## Import

In [809]:
import os
import re
import json
import pdfplumber
from langdetect import detect

## Patterns

### Personnal

In [810]:
full_name_pattern = r"^(?P<first_name>[A-Z][A-Za-zà-öø-ÿ\-']+)\s+(?P<last_name>[A-Z][a-zà-öø-ÿ\-']+)$"
headline_pattern = r"^(?:[A-Z][^\n]{3,80})$"
email_pattern = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9_-]+\.[a-zA-Z0-9._-]+"
phone_pattern = r"(?:\+?\d{1,4})?\s?(?:\d{2,3}\s?){4,5}"
location_pattern = r"(?:Abidjan|Lyon|Grand-Bassam|Anyama|Toulouse|Bingerville|Bassam)"

### Experience

In [811]:
exp_title_line_pattern = r"^(?P<title>.+?)\s*(?:[-–—|,]\s*)(?P<company>.+?)\s*\((?P<dates>[^)]+)\)"
exp_date_pattern = r"(?P<start>(?:\(?\s?[a-zà-öø-ÿA-Z.]+\s)?\d{4}|\d{2}/\d{4})\s*[à-–—]\s*(?P<end>Actuel|Présent|\w+\s?\d{4}\s?\)?)"

### Education

In [812]:
edu_title_line_pattern = r"^(?P<degree>.+?)\s*(?:[-–—|,]\s*)(?P<school>.+?)\s*\((?P<dates>[^)]+)\)"
edu_date_pattern = r"\((?P<start>\d{4})\s*[-–—]\s*(?P<end>\d{4}|En cours)\)"

### Projects

In [813]:
projects_pattern = r"\s*(?P<name>[A-Z][^—\n]+)(?:—\s*(?P<desc>.+))?"

### Skills

In [814]:
skills_pattern = r"•\s*([A-Za-zÀ-Öà-öø-ÿ\s\-/().]+)"

### Soft skills

In [815]:
soft_skills_pattern = r"(?:Communication|Leadership|Travail d'équipe|Résolution de problèmes|Gestion du temps|Adaptabilité|Pensée critique|Créativité|Esprit d'analyse|Intelligence émotionnelle|Agilité|Gestion des priorités|Assidu|Ponctuel|Rigoureux|Respectueux|Courageux|Travail en groupe ou individuellement)"

## Segmentation

In [816]:
def segment_text(text):
    sections = {
        "header": "",
        "experience": "",
        "projects": "",
        "education": "",
        "skills": "",
    }

    current_section = "header"

    for line in text.splitlines():

        line_lower = line.lower()
        if "experience" in line_lower or "expérience" in line_lower or "poste" in line_lower:
            current_section = "experience"
        elif "projets" in line_lower or "projects" in line_lower:
            current_section = "projects"
        elif "éducation" in line_lower or "études" in line_lower or "diplôme" in line_lower or "education" in line_lower or "formation" in line_lower:
            current_section = "education"
        elif "skills" in line_lower or "compétences" in line_lower:
            current_section = "skills"
            
        sections[current_section] += line + "\n"

    return sections

## Extraction functions

In [817]:
def extract_field(pattern, text, multi=False):
    pa = re.compile(pattern, flags=re.MULTILINE & re.IGNORECASE)
    if multi:
        match = []
        matches = pa.findall(text)
        for x in matches if matches else []:
            match.append(x.replace("\n", " ").strip())
    else:
        match = pa.search(text)
        match = match.group(0) if match else None
        match = match.replace("\n", " ").strip() if match else None

    return match

In [818]:
def extract_name(text):
    pa = re.compile(full_name_pattern, flags=re.MULTILINE)
    match = pa.search(text)
    if match:
        first_name = match.group('first_name').strip()
        last_name = match.group('last_name').strip()
        return first_name, last_name
    return None, None

In [819]:
def normalized_skills(skills_list):

    def clean_skill(skill):
        skill = skill.lower().strip()
        skill = re.sub(r'[\-/().]', '', skill)
        return skill

    if not skills_list:
        return None
    
    normalized = []

    SKILL_MAP = {
        "excel": ("Excel", "tool"),
        "microsoft excel": ("Excel", "tool"),
        "autocad": ("AutoCAD", "tool"),
        "pvsyst": ("PVsyst", "energy"),
        "homer": ("HOMER", "energy"),
        "python": ("Python", "language"),
        "sql": ("SQL", "language"),
        "power bi": ("Power BI", "tool"),
        "gestion de projet": ("Project Management", "management")
        }

    for skill in skills_list:
        skill_clean = clean_skill(skill)
        
        if skill_clean in SKILL_MAP:
            normalized.append(SKILL_MAP[skill_clean][0])
        
    return list(set(normalized))

In [820]:
def extract_raw_experiences_1(text):
    lines = text.split('\n')
    experiences = []
    exp_section = False

    for line in lines:
        if re.search(r'expérience|experience|poste', line, flags=re.IGNORECASE):
            exp_section = True
            continue

        if exp_section and re.search(r'formation|éducation|competence|projet', line, flags=re.IGNORECASE):
            break

        if exp_section and line.strip():
            match = re.match(exp_title_line_pattern, line)
            if match:
                title = match.group('title').strip()
                company = match.group('company').strip()
                dates = match.group('dates').strip()

                date_match = re.search(exp_date_pattern, dates)

                if date_match:
                    start_date = date_match.group('start').strip()
                    end_date = date_match.group('end').strip()
                else:
                    start_date = None
                    end_date = None

                experiences.append({
                    "title": title.strip(),
                    "company": company.strip(),
                    "date": dates.strip(),
                    "start_date": start_date,
                    "end_date": end_date,
                    "raw_title_line": line.strip()
                })

    return experiences

In [821]:
def extract_raw_experiences_2(text):
    experiences = []
    raw_experience_lines = re.findall(exp_title_line_pattern, text, flags=re.MULTILINE)
    for title, company, dates in raw_experience_lines:
        date_match = re.search(exp_date_pattern, dates)
        if date_match:
            start_date = date_match.group('start').strip()
            end_date = date_match.group('end').strip()
        else:
            start_date = None
            end_date = None

        experiences.append({
            "title": title.strip(),
            "company": company.strip(),
            "date": dates.strip(),
            "start_date": start_date,
            "end_date": end_date,
            "raw_title_line": f"{title} at {company} ({dates})"
        })
        
    return experiences

In [822]:
def extract_raw_education_1(text):
    lines = text.split('\n')
    educations = []
    edu_section = False

    for line in lines:
        if re.search(r'formation|éducation|competence|projet', line, flags=re.IGNORECASE):
            edu_section = True
            continue

        if edu_section and line.strip():
            match = re.match(edu_title_line_pattern, line)
            if match:
                degree = match.group('degree').strip()
                school = match.group('school').strip()
                dates = match.group('dates').strip()

                date_match = re.search(edu_date_pattern, dates)

                if date_match:
                    start_date = date_match.group('start').strip()
                    end_date = date_match.group('end').strip()
                else:
                    start_date = None
                    end_date = None

                educations.append({
                    "school": school.strip(),
                    "degree": degree.strip(),
                    "date": dates.strip(),
                    "start": start_date,
                    "end": end_date,
                    "raw_title_line": line.strip()
                })

    return educations

In [823]:
def extract_raw_education_2(text):
    educations = []
    raw_education_lines = re.findall(edu_title_line_pattern, text, flags=re.MULTILINE)
    for degree, school, dates in raw_education_lines:
        date_match = re.search(edu_date_pattern, dates)
        if date_match:
            start_date = date_match.group('start').strip()
            end_date = date_match.group('end').strip()
        else:
            start_date = None
            end_date = None

        educations.append({
            "school": school.strip(),
            "degree": degree.strip(),
            "date": dates.strip(),
            "start": start_date,
            "end": end_date,
            "raw_title_line": f"{degree} at {school} ({dates})"
        })
        
    return educations

## Quality

In [824]:
def completness_score(data):
    score = 0
    total_fields = 9

    if data["personal"]["first_name"]:
        score += 1
    if data["personal"]["last_name"]:
        score += 1
    if data["personal"]["headline"]:
        score += 1
    if data["personal"]["email"]:
        score += 1
    if data["personal"]["phone"]:
        score += 1
    if data["personal"]["location"]:
        score += 1

    if data["experience"]:
        score += 1

    if data["education"]:
        score += 1

    if data["skills"]:
        score += 1

    data["quality"]["completness_score"] = score / total_fields

    return data

In [825]:
def fields_missing(data):
    missing_fields = []

    if not data["personal"]["first_name"]:
        missing_fields.append("first_name")
    if not data["personal"]["last_name"]:
        missing_fields.append("last_name")
    if not data["personal"]["headline"]:
        missing_fields.append("headline")
    if not data["personal"]["email"]:
        missing_fields.append("email")
    if not data["personal"]["phone"]:
        missing_fields.append("phone")
    if not data["personal"]["location"]:
        missing_fields.append("location")
    if not data["experience"]:
        missing_fields.append("experience")
    if not data["education"]:
        missing_fields.append("education")
    if not data["skills"]:
        missing_fields.append("skills")

    data["quality"]["fields_missing"] = missing_fields
    
    return data

In [826]:
# def field_confidence(parsed_data):

## Meta

In [827]:
def detect_language(text):
    return detect(text)

## Parser

In [ ]:
def parse_cv(path):
    with pdfplumber.open(path) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()

    sections = segment_text(text)
    
    with open(f"sections_{path.split('/')[-1].rsplit('.pdf', 1)[0]}.json", "w") as f:
        json.dump(sections, f, indent=4, ensure_ascii=False)

    

    first_name, last_name = extract_name(text)

    data = {
        "personal": {
            "first_name": first_name,
            "last_name": last_name,
            "headline": extract_field(headline_pattern, text),
            "email": extract_field(email_pattern, text),
            "phone": extract_field(phone_pattern, text, multi=True),
            "location": extract_field(location_pattern, text)
        },
        "experience": extract_raw_experiences_1(text),
        "education": extract_raw_education_1(text),
        "skills": normalized_skills(extract_field(skills_pattern, text, multi=True)),
        "soft_skills": extract_field(soft_skills_pattern, text, multi=True),
        "quality": {
            "completness_score": None,
            "fields_missing": []
        },
        "meta": {
            "language": detect_language(text)
        }
    }

    data = completness_score(data)
    data = fields_missing(data)

    return data

In [829]:
# def field_confidence(parsed_data):

## Process all files

In [830]:
data_folder = "data"
pdf_files = [f for f in os.listdir(data_folder) if f.lower().endswith('.pdf')]

In [831]:
for pdf_file in pdf_files:
    pdf_path = os.path.join(data_folder, pdf_file)
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[0]
            text = page.extract_text()

        # stock text in a txt file
        with open(f"texts/output_{pdf_file.rsplit('.pdf', 1)[0]}.txt", "w") as f:
            f.write(text)
        

        # stock in a json file
        with open(f"json_files/{pdf_file.rsplit('.pdf', 1)[0]}.json", "w", encoding="utf-8") as f:
            json.dump(parse_cv(pdf_path), f, indent=4, ensure_ascii=False)
            
    except Exception as e:
        print(f"Erreur avec {pdf_file}: {e}")

Erreur avec cv_sample_5.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_8.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_3.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_9.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_4.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_1.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_2.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_6.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_7.pdf: 'tuple' object has no attribute 'replace'
Erreur avec cv_sample_10.pdf: 'tuple' object has no attribute 'replace'
